# データ・サイエンス	データ・サイエンス社会応用論 / ICT社会応用演習Ⅳ
## 13-3. 基礎自治体の人口の確認

### 1. e-Stat のデータの読み込み
#### 1-a) 下準備

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import japanize_matplotlib

In [2]:
CENSUS_POPULATION_PATH = 'data/FEH_00200521_250109010954.csv'
LG_CODE_PATH = 'data/000925835.xlsx'
TOKYO_ADMIN_PATH = 'data/shape/N03-20240101_13.geojson'

#### 1-b) DataFrameへの読み込み

In [20]:
df_population = pd.read_csv(CENSUS_POPULATION_PATH, encoding='shift-jis', skiprows=14)
df_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6228 entries, 0 to 6227
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   時間軸（年次） コード                 6228 non-null   int64  
 1   時間軸（年次） 補助コード               0 non-null      float64
 2   時間軸（年次）                     6228 non-null   object 
 3   国籍総数か日本人 コード                6228 non-null   int64  
 4   国籍総数か日本人 補助コード              0 non-null      float64
 5   国籍総数か日本人                    6228 non-null   object 
 6   男女 コード                      6228 non-null   int64  
 7   男女 補助コード                    0 non-null      float64
 8   男女                          6228 non-null   object 
 9   全国，都道府県，市区町村（人口集中地区） コード    6228 non-null   int64  
 10  全国，都道府県，市区町村（人口集中地区） 補助コード  0 non-null      float64
 11  全国，都道府県，市区町村（人口集中地区）        6228 non-null   object 
 12  /年齢                         0 non-null      float64
 13  総数                          6228 

In [21]:
df_population

,時間軸（年次） コード,時間軸（年次） 補助コード,時間軸（年次）,国籍総数か日本人 コード,国籍総数か日本人 補助コード,国籍総数か日本人,男女 コード,男女 補助コード,男女,全国，都道府県，市区町村（人口集中地区） コード,...,90〜94歳,95〜99歳,100歳以上,年齢「不詳」,（再掲）15歳未満,（再掲）15〜64歳,（再掲）65歳以上,（再掲）75歳以上,（再掲）85歳以上,（再掲）20〜69歳
0,2020000000,NaN,2020年,0,NaN,国籍総数,0,NaN,総数,0,...,983383,253476,39738,2555614,10660791,52836316,22233206,11399430,3479063,53909973
1,2020000000,NaN,2020年,0,NaN,国籍総数,0,NaN,総数,1000,...,57245,15091,2263,52478,432950,2303957,1183622,587281,194610,2419768
2,2020000000,NaN,2020年,0,NaN,国籍総数,0,NaN,総数,1100,...,25388,6645,993,30514,209256,1156977,519290,252071,84863,1205248
3,2020000000,NaN,2020年,0,NaN,国籍総数,0,NaN,総数,1101,...,3176,800,113,9537,23694,156945,56397,27876,10309,161415
4,2020000000,NaN,2020年,0,NaN,国籍総数,0,NaN,総数,1102,...,3394,846,133,4328,32538,169440,73872,35192,11313,174756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6223,2020000000,NaN,2020年,1,NaN,うち日本人,2,NaN,女,47328,...,19,14,1,149,1006,2782,399,172,68,2666
6224,2020000000,NaN,2020年,1,NaN,うち日本人,2,NaN,女,47329,...,61,16,1,6,908,3201,1285,553,187,3334
6225,2020000000,NaN,2020年,1,NaN,うち日本人,2,NaN,女,47348,...,101,40,12,6,1592,5186,1751,857,330,5229
6226,2020000000,NaN,2020年,1,NaN,うち日本人,2,NaN,女,47350,...,137,40,8,15,3361,9927,2880,1317,462,9983


#### 1-c) DataFrameの下処理

In [15]:
df_population = df_population.rename(columns={
    '全国，都道府県，市区町村（人口集中地区） コード': 'code',
    '全国，都道府県，市区町村（人口集中地区）': 'name',
    '総数': 'total'
})
df_population['total'] = df_population['total'].apply(pd.to_numeric, errors='coerce')
df_population['under19'] = df_population['0〜4歳']+df_population['5〜9歳']+df_population['10〜14歳']+df_population['15〜19歳']
df_population['ratio19'] = df_population['under19'] / df_population['total']
df_population = df_population[['code', 'name', 'total', 'under19', 'ratio19']]
df_population 

,code,name,total,under19,ratio19
0,0,全国,88285927,14633930,0.165756
1,1000,北海道,3973007,605997,0.152529
2,1100,札幌市,1916037,291473,0.152123
3,1101,札幌市中央区,246573,33266,0.134913
4,1102,札幌市北区,280178,46236,0.165024
...,...,...,...,...,...
6223,47328,中城村,4336,1262,0.291052
6224,47329,西原町,5400,1198,0.221852
6225,47348,与那原町,8535,2047,0.239836
6226,47350,南風原町,16183,4173,0.257863


### 2. 地方公共団体コードの読み込み
#### 2-a) DataFrameへの読み込み

In [28]:
df_lg = pd.read_excel(LG_CODE_PATH, dtype=str)
df_lg['団体コード'] = df_lg['団体コード'].astype(str)
df_lg

,団体コード,都道府県名\n（漢字）,市区町村名\n（漢字）,都道府県名\n（カナ）,市区町村名\n（カナ）
0,010006,北海道,NaN,ﾎｯｶｲﾄﾞｳ,NaN
1,011002,北海道,札幌市,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼ
2,012025,北海道,函館市,ﾎｯｶｲﾄﾞｳ,ﾊｺﾀﾞﾃｼ
3,012033,北海道,小樽市,ﾎｯｶｲﾄﾞｳ,ｵﾀﾙｼ
4,012041,北海道,旭川市,ﾎｯｶｲﾄﾞｳ,ｱｻﾋｶﾜｼ
...,...,...,...,...,...
1789,473618,沖縄県,久米島町,ｵｷﾅﾜｹﾝ,ｸﾒｼﾞﾏﾁｮｳ
1790,473626,沖縄県,八重瀬町,ｵｷﾅﾜｹﾝ,ﾔｴｾﾁｮｳ
1791,473758,沖縄県,多良間村,ｵｷﾅﾜｹﾝ,ﾀﾗﾏｿﾝ
1792,473812,沖縄県,竹富町,ｵｷﾅﾜｹﾝ,ﾀｹﾄﾐﾁｮｳ


In [29]:
df_lg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1794 entries, 0 to 1793
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   団体コード       1794 non-null   object
 1   都道府県名
（漢字）  1794 non-null   object
 2   市区町村名
（漢字）  1747 non-null   object
 3   都道府県名
（カナ）  1794 non-null   object
 4   市区町村名
（カナ）  1747 non-null   object
dtypes: object(5)
memory usage: 70.2+ KB


#### 2-b) DataFrameの下処理

In [30]:
df_lg = df_lg.rename(columns = {
    '団体コード': 'code',
    '都道府県名\n（漢字）': 'prefecture',
    '市区町村名\n（漢字）': 'city'
})
df_lg = df_lg[['code', 'prefecture', 'city']]
df_lg

,code,prefecture,city
0,010006,北海道,NaN
1,011002,北海道,札幌市
2,012025,北海道,函館市
3,012033,北海道,小樽市
4,012041,北海道,旭川市
...,...,...,...
1789,473618,沖縄県,久米島町
1790,473626,沖縄県,八重瀬町
1791,473758,沖縄県,多良間村
1792,473812,沖縄県,竹富町


In [31]:
df_lg['code'] = df_lg['code'].apply(lambda x: x[:5])
df_lg = df_lg[~df_lg['code'].str.endswith('000')]
df_lg

,code,prefecture,city
1,01100,北海道,札幌市
2,01202,北海道,函館市
3,01203,北海道,小樽市
4,01204,北海道,旭川市
5,01205,北海道,室蘭市
...,...,...,...
1789,47361,沖縄県,久米島町
1790,47362,沖縄県,八重瀬町
1791,47375,沖縄県,多良間村
1792,47381,沖縄県,竹富町


### 3. 国勢調査のデータと基礎自治体データの結合
#### 3-a) DataFrameの結合

In [32]:
df_lg_population = df_population.merge(df_lg, how='inner')
df_lg_population

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

#### 3-b) データの確認

In [ ]:
df_lg_population.sort_values('total', ascending=False)

In [ ]:
df_lg_population.sort_values('ratio', ascending=False)

### 4. コロプレス図の作成
#### 4-a) 行政区域データの読み込み

In [ ]:
gdf_tokyo_admin = gpd.read_file(TOKYO_ADMIN_PATH)
gdf_tokyo_admin

#### 4-b) 地方公共団体コードでグルーピング

In [ ]:
gdf_tokyo_admin_grouped = gdf_tokyo_admin.dissolve(by='N03_007')
gdf_tokyo_admin_grouped = gdf_tokyo_admin_grouped.reset_index()
gdf_tokyo_admin_grouped

#### 4-c) 国勢調査のDataFrameを結合

In [ ]:
gdf_tokyo_population = gdf_tokyo_admin_grouped.merge(df_population, left_on='N03_007', right_on='code')
gdf_tokyo_population

#### 4-d) 島嶼部の削除

In [ ]:
min_lon, min_lat = 138.0, 35.0
max_lon, max_lat = 140.0, 36.0
   
bbox_polygon = Polygon([(min_lon, min_lat), (min_lon, max_lat), (max_lon, max_lat), (max_lon, min_lat)])

In [ ]:
gdf_tokyo_population_clipped = gdf_tokyo_population[gdf_tokyo_population.within(bbox_polygon)]
gdf_tokyo_population_clipped

#### 4-e) コロプレス図の表示

In [ ]:
gdf_tokyo_population_clipped.plot(column='total', cmap='OrRd', legend=True, figsize=(20,4))

In [ ]:
gdf_tokyo_population_clipped.plot(column='ratio', cmap='OrRd', legend=True, figsize=(20,4))